# Задание 2.1 - Нейронные сети

В этом задании вы реализуете и натренируете настоящую нейроную сеть своими руками!

В некотором смысле это будет расширением прошлого задания - нам нужно просто составить несколько линейных классификаторов вместе!

<img src="https://i.redd.it/n9fgba8b0qr01.png" alt="Stack_more_layers" width="400px"/>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer
from model import TwoLayerNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD
from metrics import multiclass_accuracy

# Загружаем данные

И разделяем их на training и validation.

In [3]:
def prepare_for_neural_network(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    return train_flat, test_flat
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_neural_network(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

# Как всегда, начинаем с кирпичиков

Мы будем реализовывать необходимые нам слои по очереди. Каждый слой должен реализовать:
- прямой проход (forward pass), который генерирует выход слоя по входу и запоминает необходимые данные
- обратный проход (backward pass), который получает градиент по выходу слоя и вычисляет градиент по входу и по параметрам

Начнем с ReLU, у которого параметров нет.

In [4]:
# TODO: Implement ReLULayer layer in layers.py
# Note: you'll need to copy implementation of the gradient_check function from the previous assignment

X = np.array([[1,-2,3],
              [-1, 2, 0.1]
              ])

assert check_layer_gradient(ReLULayer(), X)

8.79719785676 [[ 1.61599406  0.          2.76252524]
 [-0.         -0.5060347  -0.94302519]]
Gradient check passed!


А теперь реализуем полносвязный слой (fully connected layer), у которого будет два массива параметров: W (weights) и B (bias).

Все параметры наши слои будут хранить через специальный класс `Param`, в котором будут храниться значения параметров и градиенты этих параметров, вычисляемые во время обратного прохода.

Это даст возможность аккумулировать (суммировать) градиенты из разных частей функции потерь, например, из cross-entropy loss и regularization loss.

In [5]:
# TODO: Implement FullyConnected layer forward and backward methods
assert check_layer_gradient(FullyConnectedLayer(3, 4), X)
# TODO: Implement storing gradients for W and B
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'W')
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'B')

-0.00418379929801 [[  4.18948095e-04   1.76337767e-03  -3.49979411e-04]
 [  7.82747412e-05  -6.12293143e-04  -1.09946675e-03]]
Gradient check passed!
0.020193253954 [[-1.67429365  2.15318718 -0.15893916 -0.56958173]
 [ 3.34858731 -4.30637437  0.31787831  1.13916345]
 [-0.36258122  5.45842018 -3.37773459 -0.89339967]]
Gradient check passed!
-0.0108456847394 [[-0.632844   -0.39002832  1.75340995  0.51081366]]
Gradient check passed!


## Создаем нейронную сеть

Теперь мы реализуем простейшую нейронную сеть с двумя полносвязным слоями и нелинейностью ReLU. Реализуйте функцию `compute_loss_and_gradients`, она должна запустить прямой и обратный проход через оба слоя для вычисления градиентов.

In [6]:
# TODO: In model.py, implement compute_loss_and_gradients function
print(train_X[:2])
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 3, reg = 0)
loss = model.compute_loss_and_gradients(train_X[:2], train_y[:2])

# TODO Now implement backward pass and aggregate all of the params
check_model_gradient(model, train_X[:2], train_y[:2])

[[-0.08975373 -0.03568431  0.08994824 ..., -0.12654745 -0.03998549
   0.09710392]
 [ 0.01612863  0.01921765  0.05465412 ..., -0.15007686 -0.13410314
  -0.10681765]]
Checking gradient for W-1
2.30062671713 [[  1.05393580e-05   0.00000000e+00  -2.23528083e-05]
 [  1.25578983e-05   0.00000000e+00  -4.99938161e-06]
 [  3.57140937e-05   0.00000000e+00   4.39014438e-05]
 ..., 
 [ -9.80687160e-05   0.00000000e+00  -8.39833186e-05]
 [ -8.76305797e-05   0.00000000e+00  -5.28532904e-05]
 [ -6.98006962e-05   0.00000000e+00  -2.94876473e-06]]
Gradient check passed!
Checking gradient for B-1
2.30062671716 [[ 0.00065346  0.          0.0006072 ]]
Gradient check passed!
Checking gradient for W-2
2.30062671109 [[ 0.00037503  0.00037535  0.00037567  0.0003749   0.0003757   0.00037455
   0.00037548  0.00037493  0.00037505 -0.00337666]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.00050123  0.00050165  0.00050209  0.00050104  0.00

True

Теперь добавьте к модели регуляризацию - она должна прибавляться к loss и делать свой вклад в градиенты.

In [7]:
# TODO Now implement l2 regularization in the forward and backward pass
model_with_reg = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 3, reg = 1e-1)
loss_with_reg = model_with_reg.compute_loss_and_gradients(train_X[:2], train_y[:2])
assert loss_with_reg > loss and not np.isclose(loss_with_reg, loss), \
    "Loss with regularization (%2.4f) should be higher than without it (%2.4f)!" % (loss, loss_with_reg)

check_model_gradient(model_with_reg, train_X[:2], train_y[:2])

Checking gradient for W-1
2.30300066005 [[ -1.97632526e-04  -6.47184905e-05  -2.32640885e-04]
 [  1.53837763e-04   6.05300258e-05   8.69655510e-05]
 [  2.26173872e-04  -1.39763073e-05   1.49505283e-04]
 ..., 
 [ -9.58146888e-05  -6.82463445e-06   1.58581489e-04]
 [  3.09991371e-04  -9.50381061e-05   2.49737747e-04]
 [  2.17393302e-04   1.89034051e-04   2.36248895e-04]]
Gradient check passed!
Checking gradient for B-1
2.3030006577 [[-0.00010136  0.00061913 -0.00110427]]
Gradient check passed!
Checking gradient for W-2
2.30300066875 [[  3.39012966e-05   2.96367547e-04  -3.31575631e-05   6.68423849e-06
    5.16626623e-04   1.66051164e-04  -1.93627095e-04   9.95912794e-05
    1.58614874e-04  -1.60238854e-04]
 [  7.86132071e-04   6.09759537e-04   9.07615761e-04   5.22062608e-04
    6.89264761e-04   5.71071104e-04   9.13211773e-04   7.78545524e-04
    9.27546318e-04  -7.99319194e-03]
 [  3.87328798e-05   3.53350809e-04  -3.44278575e-05   3.99531257e-04
    1.64681499e-04   2.79508964e-05   4

True

Также реализуем функцию предсказания (вычисления значения) модели на новых данных.

Какое значение точности мы ожидаем увидеть до начала тренировки?

In [8]:
# Finally, implement predict function!

# TODO: Implement predict function
# What would be the value we expect?
multiclass_accuracy(model_with_reg.predict(train_X[:30]), train_y[:30]) 

3  out of  30


0.10000000000000001

# Допишем код для процесса тренировки

In [9]:
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, SGD())

# TODO Implement missing pieces in Trainer.fit function
# You should expect loss to go down and train and val accuracy go up for every epoch
loss_history, train_history, val_history = trainer.fit()

1770  out of  9000
206  out of  1000
Loss: 2.327327, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.316935, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.308675, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.302105, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.296871, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.292700, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.289365, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.286691, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.284543, Train accuracy: 0.196667, val accuracy: 0.206000
1770  out of  9000
206  out of  1000
Loss: 2.282812, Train accuracy: 0.196667, val

ValueError: too many values to unpack (expected 3)

In [10]:
plt.plot(train_history)
plt.plot(val_history)

NameError: name 'train_history' is not defined

In [11]:
plt.plot(loss_history)

NameError: name 'loss_history' is not defined

# Улучшаем процесс тренировки

Мы реализуем несколько ключевых оптимизаций, необходимых для тренировки современных нейросетей.

## Уменьшение скорости обучения (learning rate decay)

Одна из необходимых оптимизаций во время тренировки нейронных сетей - постепенное уменьшение скорости обучения по мере тренировки.

Один из стандартных методов - уменьшение скорости обучения (learning rate) каждые N эпох на коэффициент d (часто называемый decay). Значения N и d, как всегда, являются гиперпараметрами и должны подбираться на основе эффективности на проверочных данных (validation data). 

В нашем случае N будет равным 1.

In [ ]:
# TODO Implement learning rate decay inside Trainer.fit method
# Decay should happen once per epoch

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, SGD(), learning_rate_decay=0.99)

initial_learning_rate = trainer.learning_rate
loss_history, train_history, val_history = trainer.fit()

assert trainer.learning_rate < initial_learning_rate, "Learning rate should've been reduced"
assert trainer.learning_rate > 0.5*initial_learning_rate, "Learning rate shouldn'tve been reduced that much!"

# Накопление импульса (Momentum SGD)

Другой большой класс оптимизаций - использование более эффективных методов градиентного спуска. Мы реализуем один из них - накопление импульса (Momentum SGD).

Этот метод хранит скорость движения, использует градиент для ее изменения на каждом шаге, и изменяет веса пропорционально значению скорости.
(Физическая аналогия: Вместо скорости градиенты теперь будут задавать ускорение, но будет присутствовать сила трения.)

```
velocity = momentum * velocity - learning_rate * gradient 
w = w + velocity
```

`momentum` здесь коэффициент затухания, который тоже является гиперпараметром (к счастью, для него часто есть хорошее значение по умолчанию, типичный диапазон -- 0.8-0.99).

Несколько полезных ссылок, где метод разбирается более подробно:  
http://cs231n.github.io/neural-networks-3/#sgd  
https://distill.pub/2017/momentum/

In [ ]:
# TODO: Implement MomentumSGD.update function in optim.py

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, MomentumSGD(), learning_rate=1e-1, learning_rate_decay=0.97)

# You should see even better results than before!
loss_history, train_history, val_history = trainer.fit()

# Ну что, давайте уже тренировать сеть!

## Последний тест - переобучимся (overfit) на маленьком наборе данных

Хороший способ проверить, все ли реализовано корректно - переобучить сеть на маленьком наборе данных.  
Наша модель обладает достаточной мощностью, чтобы приблизить маленький набор данных идеально, поэтому мы ожидаем, что на нем мы быстро дойдем до 100% точности на тренировочном наборе. 

Если этого не происходит, то где-то была допущена ошибка!

In [ ]:
data_size = 15
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X[:data_size], train_y[:data_size], val_X[:data_size], val_y[:data_size])
trainer = Trainer(model, dataset, MomentumSGD(), learning_rate=1e-1, num_epochs=150, batch_size=5)

# You should expect this to reach 1.0 training accuracy 
loss_history, train_history, val_history, learning_rate_history = trainer.fit()

Теперь найдем гипепараметры, для которых этот процесс сходится быстрее.
Если все реализовано корректно, то существуют параметры, при которых процесс сходится в **20** эпох или еще быстрее.
Найдите их!

In [ ]:
def keywithmaxval(d):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

In [ ]:
# Now, tweak some hyper parameters and make it train to 1.0 accuracy in 20 epochs or less

learning_rates = [1e-1, 1e-2, 1e-3, 1e-4]
reg_strengths = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
l_r_to_accuracy = {}

for l in learning_rates:
    for r in reg_strengths:

        model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 110, reg = r)
        # TODO: Change any hyperparamers or optimizators to reach training accuracy in 20 epochs
        dataset = Dataset(train_X[:data_size], train_y[:data_size], val_X[:data_size], val_y[:data_size])
        trainer = Trainer(model, dataset, MomentumSGD(), learning_rate=l, learning_rate_decay=0.97, num_epochs=20, batch_size=5)
        loss_history, train_history, val_history, learning_rate_history = trainer.fit()
        l_r_to_accuracy[str(l) + ', ' + str(r)] = np.average(train_history)
        print("Neural Network with l = %e, r = %e" % (l, r))
        print("Accuracy on training set: %4.2f" % (l_r_to_accuracy[str(l) + ', ' + str(r)])) 
        #plt.plot(train_history)
        #plt.plot(val_history)

key_of_best = keywithmaxval(l_r_to_accuracy)
print(keywithmaxval(l_r_to_accuracy), ' accuracy is ', (l_r_to_accuracy[key_of_best]))

# Итак, основное мероприятие!

Натренируйте лучшую нейросеть! Можно добавлять и изменять параметры, менять количество нейронов в слоях сети и как угодно экспериментировать. 

Добейтесь точности лучше **40%** на validation set.

In [ ]:
# Let's train the best one-hidden-layer network we can

learning_rates = 1e-4
reg_strength = 1e-3
learning_rate_decay = 0.999
hidden_layer_size = 128
num_epochs = 200
batch_size = 64
learning_rate_history = []
#reg = 0.01
#learning_rate=0.1

'''
BEST OF THE BEST RECOMMENDED FOR EVERY DATA SCIENTIST!!!
best_classifier = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 110, reg = 1e-6)
dataset = Dataset(train_X, train_y, val_X, val_y)
best_trainer = Trainer(best_classifier, dataset, MomentumSGD(), learning_rate=1e-1, learning_rate_decay=0.97, num_epochs=50, batch_size=64)
loss_history, train_history, val_history, learning_rate_history = best_trainer.fit()
'''

best_classifier = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 110, reg = 1e-6)
dataset = Dataset(train_X, train_y, val_X, val_y)
best_trainer = Trainer(best_classifier, dataset, MomentumSGD(), learning_rate=1e-1, learning_rate_decay=0.97, num_epochs=50, batch_size=64)
loss_history, train_history, val_history, learning_rate_history = best_trainer.fit()

best_val_accuracy = np.average(val_history)

# TODO find the best hyperparameters to train the network
# Don't hesitate to add new values to the arrays above, perform experiments, use any tricks you want
# You should expect to get to at least 40% of valudation accuracy
# Save loss/train/history of the best classifier to the variables above

print('best validation accuracy achieved: %f' % best_val_accuracy)

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(211)
plt.title("Loss")
plt.plot(loss_history)
plt.subplot(212)
plt.title("Train/validation accuracy")
plt.plot(train_history)
plt.plot(val_history)

In [ ]:
plt.title("Learning rate progress")
plt.plot(learning_rate_history)

# Как обычно, посмотрим, как наша лучшая модель работает на тестовых данных

In [ ]:
test_pred = best_classifier.predict(test_X)
test_accuracy = multiclass_accuracy(test_pred, test_y)
print('Neural net test set accuracy: %f' % (test_accuracy, ))